In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [20]:
train_feather_path = "../../data/train_test_splits/cog_train_family_tax_level.feather"
test_feather_path = "../../data/train_test_splits/cog_test_family_tax_level.feather"

train_df = pd.read_feather(train_feather_path)
val_df = pd.read_feather(test_feather_path)

train_values = train_df.iloc[:, 1:-13].values

sample_names = train_df.iloc[:, 0].values
cog_names = train_df.iloc[:, 1:-13].columns.tolist()
print(f"Number of samples = {len(sample_names)}")
print(f"Number of COGs = {len(cog_names)}")


Number of samples = 90445
Number of COGs = 12873


In [ ]:
# from cuml.manifold import TSNE  # GPU-accelerated TSNE from RAPIDS
# import cupy as cp  # GPU array handling

# # Convert your data to CuPy array (if not already on GPU)
# train_values_gpu = cp.asarray(train_values)

# # Run TSNE on GPU
# tsne_bin = TSNE(n_components=2, random_state=4, perplexity=50, learning_rate=100, n_iter=5000)
# bin_embeds_2d = tsne_bin.fit_transform(train_values_gpu)

# # Convert result back to NumPy if needed
# bin_embeds_2d = bin_embeds_2d.get()  # Moves data back to CPU


In [ ]:
tsne_bin = TSNE(n_components=2, random_state=4, perplexity=50, learning_rate=100, max_iter=5000)
bin_embeds_2d = tsne_bin.fit_transform(train_values)